# Исследование надежности заемщиков


In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [15]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, полученных от заказчика и указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.



In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**


In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных и ответы на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [31]:
child_debt = data.pivot_table(index=['children'], values=['debt'], aggfunc=['count', 'sum']) # создадим сводную таблицу, в которой отразим необходимые параметры: количество, сумму.
child_debt['percent'] = round(child_debt['sum']/child_debt['count']*100, 2) # создадим новый столбец, чтобы узнать процентное соотношение
child_debt = child_debt.sort_values(by='percent')
child_debt

,count,sum,percent
,debt,debt,
children,,,
5,9,0,0.00
0,14091,1063,7.54
3,330,27,8.18
1,4808,444,9.23
2,2052,194,9.45
4,41,4,9.76


In [32]:
child_debt = child_debt.reset_index().droplevel(1, axis=1)
child_debt

,children,count,sum,percent
0,5,9,0,0.00
1,0,14091,1063,7.54
2,3,330,27,8.18
3,1,4808,444,9.23
4,2,2052,194,9.45
5,4,41,4,9.76


**Вывод:** 

*Зависимость между количеством детей и возвратом средств есть. В процентном отношении случаи невозврата средств увеличиваются с ростом числа детей. С 7,5% до 9,7%. Исключение составляют те, у кого 5 детей. Но в данных их очень мало, вероятнее такие семьи редко обращаются за кредитами, а если обращаются, то понимают всю стпень ответственности в выплатах по кредитам. К тому же не каждый может себе позволить содержать семью с 5-ю детьми. Возможно это были семьи с высокими доходами.*

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
family_status_debt = data.pivot_table(index=['family_status'], values=['debt'], aggfunc=['count', 'sum']) # создадим сводную таблицу, в которой отразим необходимые параметры: количество, сумму.
family_status_debt['percent'] = round(family_status_debt['sum']/family_status_debt['count']*100, 2) # создадим новый столбец, чтобы узнать процентное соотношение
family_status_debt = family_status_debt.sort_values(by='percent') # отсортируем данные по возрастанию процента
family_status_debt = family_status_debt.reset_index().droplevel(1, axis=1)
family_status_debt

,family_status,count,sum,percent
0,вдовец / вдова,951,63,6.62
1,в разводе,1189,84,7.06
2,женат / замужем,12261,927,7.56
3,гражданский брак,4134,385,9.31
4,Не женат / не замужем,2796,273,9.76


**Вывод:** 

*Зависимость между семейным положением и возвратом кредитов в срок также имеется. Люди, которые состояли в браке или состоят в настоящий момент реже допускают просрочки по кредитам. В отличие от людей, которые находятся в свободных отношениях (гражданском браке) или не женаты/не замужем. Вероятно, люди в браке также более ответственны. Показатель процентного отношения у женатых и не женатых составляет 7,5% и 9,7% соответственно. Стоит отметить, что в делении на семейные категории есть дисбаланс. Возможно стоит создать только две категории: женаты или не женаты, а вдовцов, в разводе и в гражданском браке отнести по факту к не женатым. Так мы избавимся от крайне малых выборок.*

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
income_category = data.pivot_table(index=['total_income_category'], values=['debt'], aggfunc=['count', 'sum']) # создадим сводную таблицу, в которой отразим необходимые параметры: количество, сумму.
income_category['percent'] = round(income_category['sum']/income_category['count']*100, 2) # создадим новый столбец, чтобы узнать процентное соотношение
income_category = income_category.sort_values(by='percent') # отсортируем данные по возрастанию процента
income_category = income_category.reset_index().droplevel(1, axis=1)
income_category

,total_income_category,count,sum,percent
0,D,349,21,6.02
1,B,5014,354,7.06
2,A,25,2,8.00
3,C,15921,1353,8.50
4,E,22,2,9.09


**Вывод:** 

*Уровень дохода влияет на возврат кредита в срок. Самый высокий в процентном отношении показатель невозвратов именно в категории "Е" - заемщики с уровнем дохода до 30000. Однако среди заемщиков с доходом от 50001 до 200000 он едва меньше. Если в первом случае высокий процент можно оправдать малым числом выданных кредитов, людям с настолько малым доходом, то во втором случае результат связан скорее с количественным фактором. Ко второму случаю, к категории "С", относится более 75% заемщиков. Что не очень хорошо. Возможно стоит прересмотреть ценовые интервалы для определения категорий, как и стоит пересмотреть увеличение числа категорий. В целом более или менее корректно сравнить мы можем только две выборки "В" и "С", так как они включают в себя достаточно данных, по сравнению прочими малыми выборками. Но даже если сравнить только эти две категории, то общий вывод не измениться: люди с бОльшим доходом реже допускают просрочки по выплатам.*

#### Как разные цели кредита влияют на его возврат в срок?

In [34]:
purpose_category_debt = data.pivot_table(index=['purpose_category'], values=['debt'], aggfunc=['count', 'sum', 'mean']) # создадим сводную таблицу, в которой отразим необходимые параметры: количество, сумму, среднее.
purpose_category_debt['percent'] = round(purpose_category_debt['sum']/purpose_category_debt['count']*100, 2) # создадим новый столбец, чтобы узнать процентное соотношение
purpose_category_debt = purpose_category_debt.reset_index().droplevel(1, axis=1)
purpose_category_debt = purpose_category_debt.drop(columns=['mean'])
purpose_category_debt

,purpose_category,count,sum,percent
0,операции с автомобилем,4279,400,9.35
1,операции с недвижимостью,10751,780,7.26
2,получение образования,3988,369,9.25
3,проведение свадьбы,2313,183,7.91


**Вывод:** 

*Из полученых результатов можно заключить, что люди, которые берут средства для покупки жилья и проведения свадьбы - более ответственны. Это логично т.к. и первое и второе являются серьезными, ответственными шагами в жизни. После свадьбы доход семьи уже состоит из того, что заработают два человека. А приобретая недвижимость человек хочет иметь крышу над головой и собственность, которую возможно передать по наследству. Но вот приобретая автомобиль или получая образование люди менее ответственно относятся к платежам. В целом в делении данных на цели для займа также есть дисбаланс. Одна категория - операции с недвижимостью - выделяется по объему среди всех прочих.*

#### Возможные причины появления пропусков в исходных данных?

*Ответ: В столбце `total_income` ошибки скорее всего появились в результате человеческой ошибки. Невнимательный сотрудник банка или же сам заемщик, когда оставлял заявку на одобрение кредита. А в столбце `days_employed` мы скорее всего имеем дело с технологической ошибкой. Достаточно посмотреть на все данные в столбце и увидеть минусовые значения, бесконечное число цифр после запятой, хотя там должны быть указаны только дни (число дней).*

### Общий вывод.

В заключение исследования мы можем дать ответы на вопросы, которые ставили себе перед его началом:

1. _Есть ли зависимость между количеством детей и возвратом кредита в срок?  Зависимость есть. В процентном отношении случаи невозврата средств увеличиваются с ростом числа детей с 7,5% до 9,7%._
<br>

2. _Есть ли зависимость между семейным положением и возвратом кредита в срок?  Есть. Люди, которые состояли в браке или состоят в настоящий момент реже допускают просрочки по кредитам. В отличие от людей, которые находятся в свободных отношениях (гражданском браке) или не женаты/не замужем._
<br>
3. _Есть ли зависимость между уровнем дохода и возвратом кредита в срок?  Уровень дохода влияет на возврат кредита в срок. Самый высокий в процентном отношении показатель невозвратов именно в категории "Е" - заемщики с уровнем дохода до 30000._
<br>
4. _Как разные цели кредита влияют на его возврат в срок? В результате исследования можно заключить, что люди, которые берут средства для покупки жилья и проведения свадьбы - более ответственны, чем те кто берет кредит на автомобиль или получение образования. Это логично т.к. и первое и второе являются серьезными, ответственными шагами в жизни._

__В качестве общего вывода отметим, что лучшими заемщиками являются люди состоящие в браке, без детей, с уровнем дохода от 30000 и которые берут займ на покупку квартиры или проведение свадьбы.__